# Baromètre des résultats - PDF reports

## Imports

In [1]:
import json
import os
import datetime
import re

import requests
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px

from fpdf import FPDF

## Parameters

In [2]:
base_folder_path = 'https://raw.githubusercontent.com/etalab/barometre-resultats/master/frontend/static/'

ovq_data_folder_path = os.path.join(base_folder_path, 'datasets', 'ovq-data')

ovq_synthese_data_path = os.path.join(ovq_data_folder_path, 'prod', 'par_territoire', 'territoire-departemental-global-light.json')
ovq_detail_data_folder_path = os.path.join(ovq_data_folder_path, 'prod', 'par_thematique')
ovq_structure_cible_path = os.path.join(ovq_data_folder_path, 'prod', 'structure', 'structure-cible.json')
ovq_structure_families_path = os.path.join(ovq_data_folder_path, 'prod', 'structure', 'structure-families.json')

dep_taxo_url = os.path.join(base_folder_path, 'datasets', 'prod', 'taxonomies', 'departements.json')
reg_taxo_url = os.path.join(base_folder_path, 'datasets', 'prod', 'taxonomies', 'regions.json')

img_dir_path = './img/'
reports_dir_path = './reports/'

## Functions

In [3]:
def mkdir_ifnotexist(path) :
    if not os.path.isdir(path) :
        os.mkdir(path)

In [4]:
#Importing taxonomies data to make a function to get all infos on a given department
r = requests.get(dep_taxo_url)
dep_dict_list = json.loads(r.content)

r = requests.get(reg_taxo_url)
reg_dict_list = json.loads(r.content)

def get_dep_infos(dep, dep_dict_list=dep_dict_list, reg_dict_list=reg_dict_list) :
        
    dep_dict = [dict_ for dict_ in dep_dict_list if dict_['dep'] == dep][0]
    dep_name = dep_dict['libelle']
    reg = dep_dict['reg']
    
    reg_dict = [dict_ for dict_ in reg_dict_list if dict_['reg'] == reg][0]
    reg_name = reg_dict['libelle']
    
    res = dict(dep=dep,
               dep_name=dep_name,
               reg=reg,
               reg_name=reg_name
              )
    
    return res

In [5]:
def get_dep_synthese_data(dep, ovq_synthese_data) :
    for dep_dict in ovq_synthese_data :
        if dep_dict['dep'] == dep :
            res = dep_dict
            break
    return res

In [6]:
def get_dep_indicateur_synthese_data(dep, id_indicateur, ovq_synthese_data) :
    dep_dict = get_dep_synthese_data(dep, ovq_synthese_data)
    
    for ovq_dict in dep_dict['ovq'] :
        for ind_dict in ovq_dict['indicateurs'] :
            if id_indicateur in ind_dict.keys() :
                res = ind_dict[id_indicateur][0]
                break
    return res

In [7]:
def get_indicateur_structure_cible(id_indicateur, structure_cible_data) :
    for ovq_dict in structure_cible_data :
        for ind_dict in ovq_dict['indicateurs'] :
            if ind_dict['id_indicateur_fr'] == id_indicateur :
                res = ind_dict
                break
    return res

In [8]:
def get_id_ovq_from_id_indicateur(id_indicateur, structure_cible_data) :
    for ovq_dict in structure_cible_data :
        for ind_dict in ovq_dict['indicateurs'] :
            if ind_dict['id_indicateur_fr'] == id_indicateur :
                id_ovq = ovq_dict['id_ovq']
                break
    return id_ovq

In [9]:
def get_ovq_structure_cible(id_ovq, structure_cible_data) :
    for ovq_dict in structure_cible_data :
        if ovq_dict['id_ovq'] == id_ovq :
            res = ovq_dict
            break
    return res

In [10]:
def get_detail_data(id_indicateur, dep, level, structure_cible_data, base_folder_path) :
    #level can be departemental, regional or national
    
    id_ovq = get_id_ovq_from_id_indicateur(id_indicateur, structure_cible_data)
    ovq_struc = get_ovq_structure_cible(id_ovq, structure_cible_data)
    
    for source_dict in ovq_struc['odm_sources_ids'] :
        if source_dict['odm_source_level'] == level :
            source_url = source_dict['odm_source_file']
            
    full_source_url = os.path.join(base_folder_path, source_url[1:])
    
    r = requests.get(full_source_url)
    data = json.loads(r.content)
    
    if level == 'departemental' :
        for dep_dict in data :
            if dep_dict['dep'] == dep :
                res = dep_dict[id_indicateur]
                break
                
    elif level == 'regional' :
        reg = get_dep_infos(dep)['reg']
        for reg_dict in data :
            if reg_dict['reg'] == reg :
                res = reg_dict[id_indicateur]
                break
                
    elif level == 'national' :
        res = data[0][id_indicateur]
    
    return res

In [11]:
def make_pct_bullet_chart(pct, bar_color='#3D49A5', bg_color='#BBBFDF') :
    
    if pct < 0 : #To improve if inverted bullet chart is possible with plotly
        bar_color = '#ED782F'
        bg_color = '#ED782F'
    
    rounded_pct = round(pct)
    
    fig = go.Figure(go.Indicator(
        mode = "gauge",
        gauge = {'shape': "bullet", 'axis': {'visible':False, 'range': [0, 100]}, 'bgcolor':bg_color, 'bordercolor':'white', 'bar':{'thickness':1, 'color':bar_color}},
        value = rounded_pct,
        number = {'suffix':'%'},
        domain = {'x': [0, 1], 'y': [0, 1]}))
    
    fig.update_layout(height = 400, width=2400, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')

    fig.add_annotation(dict(font=dict(color='white',size=100),
                                            x=0.05,
                                            y=0.5,
                                            showarrow=False,
                                            text=str(rounded_pct)+'%',
                                            textangle=0,
                                            xanchor='left',
                                            xref="paper",
                                            yref="paper"))

    return fig

In [12]:
months_fr_dict = {
    '01':'janvier',
    '02':'février',
    '03':'mars',
    '04':'avril',
    '05':'mai',
    '06':'juin',
    '07':'juillet',
    '08':'août',
    '09':'septembre',
    '10':'octobre',
    '11':'novembre',
    '12':'décembre'
}

# yearmonth should be in '2020-01' format

def yearmonth_to_fr(yearmonth) :
    month = yearmonth[-2:]
    year = yearmonth[:4]
    
    return months_fr_dict[month] + ' ' + year

In [13]:
def reformat_number(number) :
    return '{:,}'.format(number).replace(',',' ').replace('.',',')

In [14]:
def make_detail_chart(df, chart_type) :
    
    text = ['' for i in range(len(df)-1)] + [reformat_number(list(df.sort_values('date')['value'])[-1])]
    
    if chart_type == 'histo' :
        fig = px.bar(df, x='date', y='value',
                         labels={'date':'', 'value':''}, 
                         height=400,
                         width=900,
                         color=None,
                         orientation='v',
                         text=text,
                         title=None)
        fig.update_layout(showlegend=False, uniformtext_minsize=8, uniformtext_mode='hide')
        fig.layout['xaxis'].showgrid = False
        fig.update_traces(marker_color='#00AC8C', textposition='outside', cliponaxis=False)
        fig.update_xaxes(tickangle=-45,
                         tickfont=dict(size=8),
                         tickmode = 'array',
                         tickvals = list(df['date']),
                         ticktext= list(df['date_tick']))
                        
        return fig
    
    elif chart_type == 'line' :
        fig = px.line(df, x='date', y='value',
                         labels={'date':'', 'value':''}, 
                         height=400,
                         width=750,
                         color=None,
                         orientation='v',
                         text=text,
                         title=None)
        fig.update_layout(showlegend=False)
        fig.layout['xaxis'].showgrid = False
        fig.update_traces(marker=dict(size=12, line=dict(width=2, color='white')), marker_color='#00AC8C', line_color='#00AC8C', line_width=6, textposition='top center', cliponaxis=False)
        fig.update_xaxes(tickangle=-45,
                         tickfont=dict(size=8),
                         tickmode = 'array',
                         tickvals = list(df['date']),
                         ticktext= list(df['date_tick']))
        fig.data[0].update(mode='markers+lines+text')

        return fig
    
    else :
        print('Unknown chart type.')
        return None

In [15]:
def remove_html_tags(raw_html):
    tag_regex = re.compile('<.*?>')
    return re.sub(tag_regex, '', raw_html)

In [16]:
def clean_description(text) :
    
    def drop_border_spaces(string) :
        while string.startswith(' ') :
            string = string[1:]
        while string.endswith(' ') :
            string = string[:-1]
        return string
    
    text_lines = text.splitlines()
    text_lines = [line.replace('<li>', '-') for line in text_lines]
    text_lines = [remove_html_tags(line) for line in text_lines]
    text_lines = [drop_border_spaces(line) for line in text_lines]
    text_lines = [line for line in text_lines if line != '']
    
    #Managing lists
    corr_text_lines = [text_lines[0]]
    for i in range(1, len(text_lines)) :
        if text_lines[i-1] == '-' :
            new_line = '- ' + text_lines[i]
        else :
            new_line = text_lines[i]
        corr_text_lines += [new_line]
        
    corr_text_lines = [line for line in corr_text_lines if line != '-']
    
    #Joining lines by linebreaks
    corr_text = '\n'.join(corr_text_lines)
    
    return corr_text

In [17]:
#Insert here any new character that would generate Encoding errors when exporting with FPDF
def drop_encoding_issues(text) :
    return text.replace('’', "'").replace('\u20ac',' euros').replace('œ','oe').replace('…','...').replace('–','-')

## Get data and build parameters out of it

In [18]:
#Loading OVQ synthese data
r = requests.get(ovq_synthese_data_path)
ovq_synthese_data = json.loads(r.content)

In [19]:
#Departements list
dep_list = sorted(list(set([dep_dict['dep'] for dep_dict in ovq_synthese_data])))
print(len(dep_list))

101


In [20]:
#Indicateurs list
id_indicateur_list = []
for dep_dict in ovq_synthese_data :
    for ovq_dict in dep_dict['ovq'] :
        for ind_dict in ovq_dict['indicateurs'] :
            id_indicateur_list += list(ind_dict.keys())
            
id_indicateur_list = list(set(id_indicateur_list))
print(len(id_indicateur_list))

57


In [21]:
#Loading structure data

r = requests.get(ovq_structure_cible_path)
structure_cible_data = json.loads(r.content)

r = requests.get(ovq_structure_families_path)
structure_families_data = json.loads(r.content)

## Making the graphs

In [22]:
mkdir_ifnotexist(os.path.join(img_dir_path, 'graphs'))

### Synthese gauges

In [23]:
%%time
for dep in dep_list :
    for id_indicateur in id_indicateur_list :
        data = get_dep_indicateur_synthese_data(dep, id_indicateur, ovq_synthese_data)
        pct = data['target_percentage']
        
        if pct is not None :
            fig = make_pct_bullet_chart(pct)

            if fig is not None :
                img_path = os.path.join(img_dir_path, 'graphs', 'bullet_chart-synthese-target_percentage-{}-dep_{}.png'.format(id_indicateur, dep))
                fig.write_image(img_path)

CPU times: user 28.5 s, sys: 1.52 s, total: 30 s
Wall time: 2min 21s


### Detail graphs

In [24]:
%%time
short_level_dict = {
    'dep':'departemental',
    'reg':'regional',
    'nat':'national'
}

for dep in dep_list :
    for id_indicateur in id_indicateur_list :
        
        #Get information to know what level of data to use
        synthese_data = get_dep_indicateur_synthese_data(dep, id_indicateur, ovq_synthese_data)
        level = short_level_dict[synthese_data['from_level']]
        
        #Get chart type for indicateur
        chart_type = get_indicateur_structure_cible(id_indicateur, structure_cible_data)['odm_chart_type']
        
        #Get data
        detail_data = get_detail_data(id_indicateur, dep, level, structure_cible_data, base_folder_path)
        df = pd.DataFrame(detail_data)
        df = df.sort_values('date')
        df['date_tick'] = df['date'].apply(lambda x : yearmonth_to_fr(x[:7]))
        
        #Making graphs
        fig = make_detail_chart(df, chart_type)
        img_path = os.path.join(img_dir_path, 'graphs', 'graph-detail-{}-dep_{}.png'.format(id_indicateur, dep))

        if fig is not None :
            fig.write_image(img_path)

    print('{} - Dep {} done.'.format(datetime.datetime.today(), dep))

2023-08-17 11:39:18.071612 - Dep 01 done.


2023-08-17 11:39:30.599117 - Dep 02 done.


2023-08-17 11:39:43.073732 - Dep 03 done.


2023-08-17 11:39:55.587970 - Dep 04 done.


2023-08-17 11:40:08.200526 - Dep 05 done.


...

2023-08-17 11:59:38.363585 - Dep 971 done.


2023-08-17 11:59:50.877531 - Dep 972 done.


2023-08-17 12:00:03.522509 - Dep 973 done.


2023-08-17 12:00:16.045592 - Dep 974 done.


2023-08-17 12:00:28.382807 - Dep 976 done.
CPU times: user 11min 44s, sys: 6.14 s, total: 11min 50s
Wall time: 21min 28s


## Building the reports

In [25]:
mkdir_ifnotexist(reports_dir_path)

In [26]:
global title_header
title_header = ''

In [27]:
class PDF(FPDF):
    def header(self):
        if (self.page_no() != 1 ):
            # Logo
            self.image(os.path.join(img_dir_path, 'gouv.png'), 10, 8, 45)
            # Arial bold 15
            self.cell(50)
            self.set_font('Arial', 'B', 16)
            self.cell(80, 15, title_header, 0, 1, 'A')

            # Move to the right
            # Title
            self.set_font('Arial', 'I', 9)
            self.cell(50)
            #self.cell(50, 10, subtitle_header+' - aides-entreprises.data.gouv.fr', 0, 1, 'A', link='https://aides-entreprises.data.gouv.fr/')
            self.cell(50, 10, 'www.gouvernement.fr/les-actions-du-gouvernement', 0, 1, 'A', link='https://www.gouvernement.fr/les-actions-du-gouvernement')
            # Line break  
            pdf.line(40, 38, 170, 38)
            self.ln(7)
        
    # Page footer
    def footer(self):
        if (self.page_no() != 1 ):
            # Position at 1.5 cm from bottom
            self.set_y(-15)
            # Arial italic 8
            self.set_font('Arial', 'I', 8)
            # Page number
            
            self.cell(0, 10, 'Direction Interministérielle du Numérique (DINUM) - Page ' + str(self.page_no()) + '/{nb}', 0, 0, 'C')

In [28]:
def insert_indicateur(pdf, dep, ind_data, ind_struc) :
    
    #Getting relevant information in data
    initial_value = ind_data['initial_value']
    initial_value_date = ind_data['initial_value_date']
    latest_value = ind_data['latest_value']
    latest_value_date = ind_data['latest_value_date']
    target_value = ind_data['target']
    target_percentage = ind_data['target_percentage']
    target_value_date = ind_data['target_date']
    progression_percentage = ind_data['progression_percentage']
    from_level = ind_data['from_level']
    
    #Getting relevant information in structure
    id_indicateur = ind_struc['id_indicateur_fr']
    nom_indicateur = drop_encoding_issues(ind_struc['nom_indicateur'])
    ind_text_url = drop_encoding_issues(ind_struc['odm_text'])
    kpi_format = ind_struc['odm_kpi_format']
    kpi_unit = drop_encoding_issues(ind_struc['odm_kpi_unit'])
    chart_unit = drop_encoding_issues(ind_struc['odm_chart_unit'])
    to_round = ind_struc['toRound']
    
    #Numbers formating
    if to_round is not None :
        if initial_value is not None :
            initial_value = round(initial_value, to_round)
            initial_value = reformat_number(initial_value)
        if latest_value is not None :
            latest_value = round(latest_value, to_round)
            latest_value = reformat_number(latest_value)
        if target_value is not None :
            target_value = round(target_value, to_round)
            target_value = reformat_number(target_value)
            
    if progression_percentage is not None :
        if progression_percentage < 0 :
            progression_percentage_str = '- '+str(-progression_percentage).replace('.',',')+' %'
        else :
            progression_percentage_str = '+ '+str(progression_percentage).replace('.',',')+' %'
    else :
        progression_percentage_str = ''
    
    if kpi_unit.startswith('%') :
        if initial_value is not None :
            initial_value += ' %'
        if latest_value is not None :
            latest_value += ' %'
        if target_value is not None :
            target_value += ' %'
        kpi_unit = kpi_unit.replace('% ','').replace('%','')
    
    #Initializing blocks parameters
    nb_blocks = 1
    ini_block = False
    tar_block = False
    
    #Current value (always present)
    latest_value_date_str = yearmonth_to_fr(latest_value_date[:7])
    latest_value_str = str(latest_value).replace('.',',')
    
    #Insert initial block if init value exists
    if initial_value is not None :
        if float(initial_value.replace(',','.').replace(' ','').replace('%','')) > 0 :
            initial_value_date_str = yearmonth_to_fr(initial_value_date[:7])
            initial_value_str = str(initial_value).replace('.',',')
            nb_blocks += 1
            ini_block = True
        
    #Insert target block if target value exists
    if target_value is not None :
        target_value_date_str = yearmonth_to_fr(target_value_date[:7])
        target_value_str = str(target_value).replace('.',',')
        nb_blocks += 1
        tar_block = True
        
    #Blocks settings depending on existing data
    block_spacing = 5
    block_width = (182 - (nb_blocks-1)*block_spacing)/nb_blocks
    
    #INDICATOR DISPLAY
    
    #Indicator title
    pdf.set_text_color(0,0,0)
    pdf.set_font('Arial', 'B', 12)
    pdf.multi_cell(182, 5, 'Indicateur : '+ nom_indicateur, align='L')
    pdf.ln(3)
    
    #Indicator description
    full_ind_text_url = os.path.join(base_folder_path, ind_text_url[1:])
    r = requests.get(full_ind_text_url)
    ind_text = drop_encoding_issues(r.text)
    ind_text = clean_description(ind_text)
    
    pdf.set_font('Arial', '', 10)
    pdf.multi_cell(182, 5, ind_text, align='J')
    pdf.ln(3)
    
    #Data geo level indication
    if from_level == 'dep' :
        from_level_str = "Les résultats pour le département : " + get_dep_infos(dep)['dep_name']
    elif from_level == 'reg' :
        from_level_str = "Données uniquement disponibles pour la région : " + get_dep_infos(dep)['reg_name']
    elif from_level == 'nat' :
        from_level_str = "Données uniquement disponibles pour la France"
    
    pdf.set_text_color(0,0,0)
    pdf.set_font('Arial', '', 10)
    pdf.set_fill_color(245,245,245);
    pdf.cell(182, 8, from_level_str, 0, 2, 'L', fill=True)
    pdf.ln(3)
    
    #Indicator blocks
    
    y_blocks_top = pdf.get_y()
    x_blocks_left = pdf.get_x()
    
    #Initial value block
    if ini_block == True :
        pdf.set_fill_color(253, 244, 242)
        pdf.set_font('Arial', '', 8)
        pdf.cell(block_width, 8, 'En {}'.format(initial_value_date_str), 0, 1, 'L', fill=True)
        pdf.set_fill_color(254, 249, 248)
        pdf.set_font('Arial', 'B', 20)
        pdf.cell(block_width, 10, initial_value_str, 0, 1, 'L', fill=True)
        pdf.set_font('Arial', '', 10)
        pdf.cell(block_width, 3, kpi_unit, 0, 1, 'L', fill=True)
        block_left_x = pdf.get_x()
        block_bottom_y = pdf.get_y()
        y_blocks_top = block_bottom_y - (8+10+3) #Just in case there is an auto page break that puts blocks in different y
        pdf.cell(block_width, 10, '', 0, 1, 'L', fill=True)
    
    #Current value block (always here)
    if ini_block == True :
        pdf.set_xy(block_width + pdf.l_margin + block_spacing, y_blocks_top)
    pdf.set_fill_color(234, 244, 239)
    pdf.set_font('Arial', '', 8)
    pdf.cell(block_width, 8, 'En {}'.format(latest_value_date_str), 0, 2, 'L', fill=True)
    pdf.set_fill_color(244, 250, 247)
    pdf.set_font('Arial', 'B', 20)
    pdf.cell(block_width, 10, latest_value_str, 0, 2, 'L', fill=True)
    pdf.set_font('Arial', '', 10)
    pdf.cell(block_width, 3, kpi_unit, 0, 2, 'L', fill=True)
    block_left_x = pdf.get_x()
    block_bottom_y = pdf.get_y()
    y_blocks_top = block_bottom_y - (8+10+3) #Just in case there is an auto page break that puts blocks in different y
    pdf.cell(block_width, 10, progression_percentage_str, 0, 2, 'L', fill=True)
    
    #Target value block
    if tar_block == True :
        if nb_blocks == 2 :
            pdf.set_xy(block_width + pdf.l_margin + block_spacing, y_blocks_top)
        elif nb_blocks == 3 :
            pdf.set_xy(2*block_width + pdf.l_margin + 2*block_spacing, y_blocks_top)
            
        pdf.set_fill_color(229, 229, 243)
        pdf.set_font('Arial', '', 8)
        pdf.cell(block_width, 8, 'Cible {}'.format(target_value_date_str), 0, 2, 'L', fill=True)
        pdf.set_fill_color(242, 242, 248)
        pdf.set_font('Arial', 'B', 20)
        pdf.cell(block_width, 10, target_value_str, 0, 2, 'L', fill=True)
        pdf.set_font('Arial', '', 10)
        pdf.cell(block_width, 3, kpi_unit, 0, 2, 'L', fill=True)
        
        #Inserting bullet chart (prog %) if exists
        block_left_x = pdf.get_x()
        block_bottom_y = pdf.get_y()
        pdf.cell(block_width, 10, '', 0, 2, 'L', fill=True)
        bullet_chart_width = 55
        pdf.set_xy(block_left_x + block_width/2 - bullet_chart_width/2, block_bottom_y)
        try :
            prog_img_path = os.path.join(img_dir_path, 'graphs', 'bullet_chart-synthese-target_percentage-{}-dep_{}.png'.format(id_indicateur, dep))
            pdf.image(prog_img_path, w=bullet_chart_width)
        except :
            pass
    
    #To go properly on next line (under blocks)
    pdf.set_xy(block_left_x, block_bottom_y)
    pdf.cell(0, 10, '', 0, 1) #Just to go to next line
    
    #CHART DISPLAY
    try :
        chart_width = 140
        chart_img_path = os.path.join(img_dir_path, 'graphs', 'graph-detail-{}-dep_{}.png'.format(id_indicateur, dep))
        pdf.set_x(x_blocks_left + 182/2 - chart_width/2)
        pdf.image(chart_img_path, w=chart_width)

        pdf.set_x(x_blocks_left)
        pdf.set_font('Arial', 'I', 8)
        pdf.cell(182, 8, 'Unité : {}'.format(chart_unit), 0, 2, 'C')
    except :
        print('No chart found for {}'.format(id_indicateur))

    return pdf

In [29]:
%%time

mkdir_ifnotexist(os.path.join(reports_dir_path, 'pdf'))
mkdir_ifnotexist(os.path.join(reports_dir_path, 'pdf', 'par_departement'))

for dep in dep_list :
    
    pdf = PDF()
    pdf.alias_nb_pages()
    pdf.add_page()

    #Logos
    pdf.image(os.path.join(img_dir_path, 'gouv.png'), 10, 8, 125)
    
    # Arial bold 15
    pdf.set_font('Arial', 'B', 28)
    # Move to the right
    pdf.cell(50)
    pdf.ln(100)
    
    # Title
    pdf.cell(10)
    pdf.cell(50, 10, 'BAROMÈTRE DES RÉSULTATS DE', 0, 1, 'A')
    pdf.ln(10)
    pdf.cell(10)
    pdf.cell(50, 10, "L'ACTION PUBLIQUE", 0, 1, 'A')
    pdf.ln(10)
    pdf.cell(60)

    pdf.set_font('Arial', 'I', 20)
    pdf.ln(10)
    pdf.cell(10)
    pdf.cell(50, 10, 'Présentation des résultats pour le département :', 0, 1, 'A')
    pdf.ln(10)
    pdf.cell(10)
    pdf.cell(50, 10, dep + ' - ' + get_dep_infos(dep)['dep_name'], 0, 1, 'A')
    pdf.ln(70)

    pdf.set_font('Arial', 'I', 8)

    pdf.cell(10)
    pdf.cell(50, 10, "Données issues du baromètre des résultats consultable sur https://www.gouvernement.fr/les-actions-du-gouvernement", 0, 1, 'A')
    
    #Indicateurs
    
    pdf.set_left_margin(14)
    pdf.set_right_margin(14)
    
    for thematique_dict in structure_families_data :
    
        thematique = thematique_dict['nom_ovq']
        
        #Downloading thematique image
        thematique_img_url = os.path.join(base_folder_path, thematique_dict['odm_image'][1:])
        thematique_img_local_path = './img/' + thematique_img_url.split('/')[-1]
        r = requests.get(thematique_img_url)
        with open(thematique_img_local_path, 'wb') as f:
            f.write(r.content)
        
        #Section page
        title_header = ''
        pdf.add_page()
        pdf.image(thematique_img_local_path, x=0, y=70, w=210)
        pdf.ln(190)
        pdf.set_font('Arial', 'B', 25)
        pdf.cell(182, 6, thematique.upper())
        
        title_header = thematique.upper()

        id_ovq_list = thematique_dict['id_ovq']
        for id_ovq in id_ovq_list :

            try :
                ovq_dict = get_ovq_structure_cible(id_ovq, structure_cible_data)
            except :
                print('No structure cible for {}'.format(id_ovq))
                ovq_dict = None
            
            if ovq_dict is not None :
                pdf.add_page()
                pdf.ln(3)
                nom_ovq = drop_encoding_issues(ovq_dict['nom_ovq'])
                ovq_text_url = os.path.join(base_folder_path, ovq_dict['odm_text'][1:])
                r = requests.get(ovq_text_url)
                ovq_text = drop_encoding_issues(r.text)
                ovq_text = clean_description(ovq_text)
                
                #Display OVQ title
                pdf.set_font('Arial', 'B', 16)
                pdf.multi_cell(182, 6, nom_ovq, align='L')
                pdf.ln(3)
                #Display OVQ description
                pdf.set_font('Arial', '', 10)
                pdf.multi_cell(182, 5, ovq_text, align='J')
                pdf.ln(3)

                ovq_id_indicateur_list = [ind_dict['id_indicateur_fr'] for ind_dict in ovq_dict['indicateurs']]

                add_page_for_ind = False
                for id_indicateur in ovq_id_indicateur_list :
                    if add_page_for_ind == True :
                        pdf.add_page()
                    ind_data = get_dep_indicateur_synthese_data(dep, id_indicateur, ovq_synthese_data)
                    ind_struc = get_indicateur_structure_cible(id_indicateur, structure_cible_data)

                    pdf = insert_indicateur(pdf, dep, ind_data, ind_struc)
                    add_page_for_ind = True
    
    dep_name = get_dep_infos(dep)['dep_name']
    pdf.output(os.path.join(reports_dir_path, 'pdf', 'par_departement', 'Baromètre_résultats_'+dep_name+'.pdf'), 'F')
    
    print(str(datetime.datetime.today()) + ' - ' + dep + '-' + dep_name + ' done.')

2023-08-17 12:01:48.159365 - 01-Ain done.


2023-08-17 12:02:49.888446 - 02-Aisne done.


2023-08-17 12:03:54.468866 - 03-Allier done.


2023-08-17 12:04:56.737065 - 04-Alpes-de-Haute-Provence done.


2023-08-17 12:05:58.794145 - 05-Hautes-Alpes done.


...

2023-08-17 13:43:32.888115 - 971-Guadeloupe done.


2023-08-17 13:44:34.387443 - 972-Martinique done.


2023-08-17 13:45:35.463394 - 973-Guyane done.


2023-08-17 13:46:36.157428 - 974-La Réunion done.


2023-08-17 13:47:42.783050 - 976-Mayotte done.
CPU times: user 1h 40min 18s, sys: 45.1 s, total: 1h 41min 4s
Wall time: 1h 47min 14s


## Compress to ZIP